In [1]:
import numpy as np
import pandas as pd
import sqlite3 as sql
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact # para análisis interactivo
from sklearn import neighbors # basado en contenido un solo producto consumido
import joblib

from surprise import Reader, Dataset
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise.model_selection import train_test_split

# conectar base de datos
conn = sql.connect('Data/movies2.db')
cur = conn.cursor()

# ver tablas disponibles en base de datos ###
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
cur.fetchall()

[('ratings',),
 ('movies',),
 ('reco',),
 ('usuarios_sel',),
 ('movies_sel',),
 ('movies_m',),
 ('ratings_final',),
 ('full_ratings',),
 ('f_ratings',)]

In [2]:
df = pd.read_sql("SELECT * FROM f_ratings", conn)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45425 entries, 0 to 45424
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   user_id       45425 non-null  int64  
 1   movie_id      45425 non-null  int64  
 2   rating        45425 non-null  float64
 3   timestamp     45425 non-null  int64  
 4   movie_title   45425 non-null  object 
 5   movie_genres  45425 non-null  object 
 6   clean_title   45425 non-null  object 
 7   movie_year    45425 non-null  int64  
 8   fecha_nueva   45425 non-null  object 
dtypes: float64(1), int64(4), object(4)
memory usage: 3.1+ MB


In [3]:
df

,user_id,movie_id,rating,timestamp,movie_title,movie_genres,clean_title,movie_year,fecha_nueva
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,2000-07-30
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men,1995,2000-07-30
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller,Heat,1995,2000-07-30
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,Seven,0,2000-07-30
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,"Usual Suspects, The",1995,2000-07-30
...,...,...,...,...,...,...,...,...,...
45420,609,589,3.0,847220990,Terminator 2: Judgment Day (1991),Action|Sci-Fi,Terminator 2: Judgment Day,1991,1996-11-05
45421,609,590,4.0,847220802,Dances with Wolves (1990),Adventure|Drama|Western,Dances with Wolves,1990,1996-11-05
45422,609,592,3.0,847220802,Batman (1989),Action|Crime|Thriller,Batman,1989,1996-11-05
45423,609,786,3.0,847221025,Eraser (1996),Action|Drama|Thriller,Eraser,1996,1996-11-05


# <b>3. Sistema de recomendación filtro colaborativo</b>
Se identifican usuarios que han calificado las mismas películas que el usuario en las últimas semanas. Se recomiendan las películas vistas recientemente por estos usuarios.<br>
Este sistema identifica usuarios que han calificado las mismas películas que el usuario principal durante las últimas semanas. A partir de esa coincidencia, se recomiendan las 10 películas que esos usuarios han visto recientemente y que el usuario aún no ha explorado. Al utilizar la actividad reciente de otros usuarios con gustos similares, el sistema ofrece recomendaciones más personalizadas y adaptadas a las preferencias actuales del usuario.<br> La frecuencia de actualización es semanal, asegurando que las recomendaciones reflejen las últimas interacciones tanto del usuario principal como de los usuarios que comparten preferencias similares. Además, el sistema se ajusta automáticamente para excluir las películas que el usuario ya ha visto.

In [4]:
ratings = pd.read_sql("SELECT * FROM f_ratings where rating>0", conn)

# los datos deben ser leídos en un formato especial para surprise
reader = Reader(rating_scale=(0.5,5))   # la escala de la calificación
# las columnas deben estar en orden estándar: user item rating
data = Dataset.load_from_df(ratings[['user_id','movie_id','rating']], reader)

In [5]:
# Lista con los modelos
models = [KNNBasic(),KNNWithMeans(),KNNWithZScore(),KNNBaseline()]
results = {}

# for para probar varios modelos
model = models[1]
for model in models:
    CV_scores = cross_validate(model, data, measures=["MAE","RMSE"], cv=5, n_jobs=-1)
    result = pd.DataFrame.from_dict(CV_scores).mean(axis=0).\
             rename({'test_mae':'MAE', 'test_rmse': 'RMSE'})
    results[str(model).split("algorithms.")[1].split("object ")[0]] = result

performance_df = pd.DataFrame.from_dict(results).T
performance_df.sort_values(by='RMSE')

,MAE,RMSE,fit_time,test_time
knns.KNNBaseline,0.643408,0.845762,0.303393,2.224558
knns.KNNWithZScore,0.646522,0.852952,0.638523,2.479312
knns.KNNWithMeans,0.651632,0.855040,0.332330,1.988135
knns.KNNBasic,0.690430,0.907482,0.419525,1.748754


<i>KNNBaseline</i>: calcula el desvío de cada calificación con respecto al promedio y con base en esos calculan la ponderación,<br>se ecoge este modelo ya que tiene el menor valor tanto en MAE como en RMSE, lo que indica que, en promedio, este modelo realiza las predicciones con menor error en comparación con los otros modelos.

In [6]:
# Afinamiento de hiperparámentros
param_grid = {'sim_options' : {'name': ['msd','cosine'], \
                                'min_support': [5,2], \
                                'user_based': [False, True]}}

In [7]:

gridsearchKNNWithMeans = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse'], \
                                      cv=2, n_jobs=-1)
                                    
gridsearchKNNWithMeans.fit(data)

gs_model = gridsearchKNNWithMeans.best_estimator['rmse'] # mejor estimador de gridsearch


In [8]:
gridsearchKNNWithMeans.best_params["rmse"]

{'sim_options': {'name': 'msd', 'min_support': 2, 'user_based': False}}

In [9]:
gridsearchKNNWithMeans.best_score["rmse"]

0.8621834391499144

In [10]:
# Entrenar con todos los datos y Realizar predicciones con el modelo afinado
trainset = data.build_full_trainset() # esta función convierte todos los datos en entrnamiento, las funciones anteriores dividen  en entrenamiento y evaluación
model = gs_model.fit(trainset) # se reentrena sobre todos los datos posibles (sin dividir)
predset = trainset.build_anti_testset() # crea una tabla con todos los usuarios y las peliculas que no han visto

Computing the msd similarity matrix...
Done computing similarity matrix.


In [11]:
predictions = model.test(predset) # función muy pesada, hace las predicciones de rating para todos las películas que no haya leído un usuario
# la funcion test recibe un test set constriuido con build_test method, o el que genera crosvalidate

predictions_df = pd.DataFrame(predictions) # esta tabla se puede llevar a una base donde estarán todas las predicciones
predictions_df
     

,uid,iid,r_ui,est,details
0,1,31,3.696478,4.177298,"{'actual_k': 40, 'was_impossible': False}"
1,1,849,3.696478,3.670689,"{'actual_k': 40, 'was_impossible': False}"
2,1,914,3.696478,4.667909,"{'actual_k': 40, 'was_impossible': False}"
3,1,1093,3.696478,4.170720,"{'actual_k': 40, 'was_impossible': False}"
4,1,1263,3.696478,4.431743,"{'actual_k': 40, 'was_impossible': False}"
...,...,...,...,...,...
364790,609,6754,3.696478,3.377824,"{'actual_k': 22, 'was_impossible': False}"
364791,609,1172,3.696478,3.951703,"{'actual_k': 24, 'was_impossible': False}"
364792,609,8807,3.696478,3.369369,"{'actual_k': 22, 'was_impossible': False}"
364793,609,45720,3.696478,3.222339,"{'actual_k': 21, 'was_impossible': False}"


In [12]:
predictions_df['r_ui'].unique() # promedio de ratings

array([3.69647771])

In [13]:
predictions_df.sort_values(by='est',ascending=False) # se ordena en función del estimador

,uid,iid,r_ui,est,details
80067,122,904,3.696478,5.0,"{'actual_k': 40, 'was_impossible': False}"
80068,122,908,3.696478,5.0,"{'actual_k': 40, 'was_impossible': False}"
100296,154,5618,3.696478,5.0,"{'actual_k': 22, 'was_impossible': False}"
329773,543,1035,3.696478,5.0,"{'actual_k': 40, 'was_impossible': False}"
170458,276,3681,3.696478,5.0,"{'actual_k': 30, 'was_impossible': False}"
...,...,...,...,...,...
1028,3,2701,3.696478,0.5,"{'actual_k': 19, 'was_impossible': False}"
1362,3,1562,3.696478,0.5,"{'actual_k': 18, 'was_impossible': False}"
99829,153,6157,3.696478,0.5,"{'actual_k': 40, 'was_impossible': False}"
24011,36,1882,3.696478,0.5,"{'actual_k': 35, 'was_impossible': False}"


In [14]:
# función para recomendar las 10 películas con mejores predicciones y llevar base de datos para consultar resto de información
def recomendaciones(user_id,n_recomend=10):
    predictions_userID = predictions_df[predictions_df['uid'] == user_id].\
                    sort_values(by="est", ascending = False).head(n_recomend)
    recomendados = predictions_userID[['iid','est']]
    recomendados.to_sql('reco',conn,if_exists="replace") 
    recomendados = pd.read_sql('''
      SELECT DISTINCT a.iid, a.est, b.clean_title 
      FROM reco a 
      LEFT JOIN f_ratings b 
      ON a.iid = b.movie_id
      ''', conn)
    return(recomendados)


In [15]:
recomendaciones(1,10)

,iid,est,clean_title
0,1172,5.000000,Cinema Paradiso
1,1272,5.000000,Patton
2,1252,5.000000,Chinatown
3,1228,5.000000,Raging Bull
4,912,5.000000,Casablanca
5,318,5.000000,"Shawshank Redemption, The"
6,3681,4.973697,For a Few Dollars More
7,1266,4.971846,Unforgiven
8,1233,4.968931,"Boot, Das"
9,1242,4.963114,Glory
